In [37]:
import pandas as pd
import re
import glob
# import math
# import string

from math import nan
from string import digits

Loading data

In [104]:
# df = pd.read_csv("RESINTEL_ENERGYSOLUTIONS_2_BUILDINGPERMIT_0002.txt", delimiter='\t',encoding='latin-1')
df = pd.read_csv('Permit Mining Results Sept2024.csv')
columns = list(df)
df.info()

/var/folders/5h/45mckv054hbfvkst9pks_x800000gn/T/ipykernel_90805/2246647576.py:2: DtypeWarning: Columns (6,10,12,17,19,21,22,23,24,25,26,27,28,29,30,33,34,44,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Permit Mining Results Sept2024.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16444228 entries, 0 to 16444227
Data columns (total 94 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   Unnamed: 0                                  int64  
 1   BuildingPermitID                            int64  
 2   [ATTOM ID]                                  float64
 3   CountyName                                  object 
 4   StateCountyFIPS                             float64
 5   PropertyAddressFull                         object 
 6   PropertyAddressAddressHouseNumber           object 
 7   PropertyAddressAddressStreetDirection       object 
 8   PropertyAddressAddressStreetName            object 
 9   PropertyAddressAddressStreetSuffix          object 
 10  PropertyAddressAddressStreetPostDirection   object 
 11  PropertyAddressAddressUnitPrefix            object 
 12  PropertyAddressAddressUnit                  object 
 13  PropertyAddressCity      

In [105]:
df = df.drop(columns=['combined_column', 'Existing Amp', 'Upgrade Amp'])

In [106]:
# formatting description for mining
df['Description'] = df['Description'].astype(str)
df['Description'] = df['Description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

The next cell is big but fairly straightforward. It contains the word lists for mining different upgrades as well as all the necessary functions. However, it is built around two versions, one which is the full mining process from the older permit mining code, while the other is only for mining upgrades I added.

In [84]:
# all upgrade word pairs
product_categories = {
    'Solar PV': ['solar', 'pv', 'photovoltaic', 'photo voltaic', 'solar panel', 'solar module','inverter','rooftop','ground mount','optimizer'],
    'Battery storage': ['battery', 'energy storage', 'storage system','powerwall','LG Chem','batt','enphase'],
    'Electric Vehicle Charger': ['ev charger', 'evse', 'electric vehicle charger', 'electric car charger',
                                'charger','evc','electric vehicle','level 2','charge','charging','station', 'ev'],
    'Electrical Panel Upgrade': ['panel upgrade', 'circuit breaker','load center', 'electrical service upgrade', 'service panel',
                                'breaker','subpanel','derate','mpu','msp','main panel'],
    'Transformer': ['transformer', 'power transformer', 'distribution transformer'],
    'Water Heater': ['water heater'],
    'Cool Roof': ['cool roof'],
    'Kitchen Remodel': ['kitchen remodel', 'remodel kitchen', 'kitchen bathroom remodel'],
    'Bathroom Remodel': ['bathroom remodel', 'remodel bathroom', 'restroom remodel', 'remodel restroom',
                         'bathroom kitchen remodel', 'ada restroom', 'ada bathroom'],
    'AC': ['hvac', 'air conditioning'],
    'Reroof': ['reroof', 'roof install', 'roof replacement'],
    'Spa/Pool': ['spa', 'pool'],
    'Addition': ['sqft addition', 'square foot addition', 'room addition', 'story addition', 'attach garage', 'new garage'],
    'Lighting Install': ['install light', 'light fixtures'],
    'Other Remodel': ['interior remodel', 'bedroom remodel', 'bedroom bathroom remodel', 'remodel interior',
                      'remodel bedroom', 'remodel bathroom bedroom'],
    'Wall Heater': ['wall heater', 'wall furnace', 'block wall'],
    'Tankless Water Heater': ['tankless'],
    'Fire Damage': ['fire damage', 'burn damage', 'fire repair'],
    'Windows': ['window size', 'windows size', 'window install', 'windows install', 'fenestration', 'retrofit windows'],
    'Smoke Detectors': ['smoke detectors', 'fire alarm'],
    'Carbon Monoxide Detector': ['carbon monoxide', 'co detector', 'monoxide detector'],
    'Ductwork': ['duct'],
    'Gas Furnace': ['gas furnace', 'gas line', 'install furnace'],
    'Dishwasher': ['dishwasher replacement', 'new dishwasher', 'dishwasher install', 'install dishwasher'],
    'Washing Machine': ['washer dryer', 'washer/dryer'],
    'Dryer': ['dryer replacement', 'new dryer', 'dryer install', 'washer dryer', 'washer/dryer'],
    'Elevator': ['elevator'],
    'Skylight': ['install skylight', 'skylight addition', 'roof skylight']

}

# upgrade word pairs only for new upgrades
new_product_categories = {
    'Water Heater': ['water heater'],
    'Cool Roof': ['cool roof'],
    'Kitchen Remodel': ['kitchen remodel', 'remodel kitchen', 'kitchen bathroom remodel'],
    'Bathroom Remodel': ['bathroom remodel', 'remodel bathroom', 'restroom remodel', 'remodel restroom',
                         'bathroom kitchen remodel', 'ada restroom', 'ada bathroom'],
    'AC': ['hvac', 'air conditioning'],
    'Reroof': ['reroof', 'roof install', 'roof replacement'],
    'Spa/Pool': ['spa', 'pool'],
    'Addition': ['sqft addition', 'square foot addition', 'room addition', 'story addition', 'attach garage', 'new garage'],
    'Lighting Install': ['install light', 'light fixtures'],
    'Other Remodel': ['interior remodel', 'bedroom remodel', 'bedroom bathroom remodel', 'remodel interior',
                      'remodel bedroom', 'remodel bathroom bedroom'],
    'Wall Heater': ['wall heater', 'wall furnace', 'block wall'],
    'Tankless Water Heater': ['tankless'],
    'Fire Damage': ['fire damage', 'burn damage', 'fire repair'],
    'Windows': ['window size', 'windows size', 'window install', 'windows install', 'fenestration', 'retrofit windows'],
    'Smoke Detectors': ['smoke detectors', 'fire alarm'],
    'Carbon Monoxide Detector': ['carbon monoxide', 'co detector', 'monoxide detector'],
    'Ductwork': ['duct'],
    'Gas Furnace': ['gas furnace', 'gas line', 'install furnace'],
    'Dishwasher': ['dishwasher replacement', 'new dishwasher', 'dishwasher install', 'install dishwasher'],
    'Washing Machine': ['washer dryer', 'washer/dryer'],
    'Dryer': ['dryer replacement', 'new dryer', 'dryer install', 'washer dryer', 'washer/dryer'],
    'Elevator': ['elevator'],
    'Skylight': ['install skylight', 'skylight addition', 'roof skylight']
}

test_categories = {
    'Electrical Panel Upgrade': ['panel upgrade', 'circuit breaker','load center', 'electrical service upgrade', 'service panel',
                                'breaker','subpanel','derate','mpu','msp','main panel']
}

# OLD FUNCTIONS START HERE
def classify_product(description):
    categories = product_categories
    if isinstance(description, float):
        description = str(description)
    # initialize a dictionary to store the boolean flags for each product category in the description
    category_flags = {category: False for category in categories}

    # check if any of the keywords for each product category is present in the description
    for category, keywords in categories.items():
        for keyword in keywords:
            if keyword.lower() in description.lower():
                category_flags[category] = True

    # check if "A" is present together with a number greater than 100 in the same sentence
    if re.search(r'\b(\d{3,}A)\b', description, re.IGNORECASE):
        category_flags['Electrical Panel Upgrade'] = True
    # check if 'kW' is present along with a number or as is
    if re.search(r'\b(\d+\.?\d*)\s*kW\b', description, re.IGNORECASE) and ('panel' in description.lower() or 'solar' in description.lower()):
        category_flags['Solar PV'] = True
        # check if 'ess' is present for 'Battery storage' category
    if re.search(r'\bess\b', description, re.IGNORECASE):
        category_flags['Battery storage'] = True

    # check if 'panel' appears in the description of 'Solar PV' category and ensure 'solar' is also present
    if 'panel' in description.lower() and 'solar' in description.lower():
        category_flags['Solar PV'] = True
    if 'panel' in description.lower() and 'amp' in description.lower():
        category_flags['Electrical Panel Upgrade'] = True
        # check if 'panel' appears in the description of 'Solar PV' category and ensure 'solar' is also present
    if 'tesla' in description.lower() and 'batt' in description.lower():
        category_flags['Battery storage'] = True

# check if 'tesla' appears and ensure 'charger' is also present for 'Electric Vehicle Charger' category
    if 'tesla' in description.lower() and ('station' in description.lower() or 'electric' in description.lower() or 'ev' in description.lower() or 'charge' in description.lower() or 'vehicle' in description.lower()):
        category_flags['Electric Vehicle Charger'] = True
    # check if 'derate' is present in the description
    if 'derate' in description.lower():
        category_flags['derate'] = True
    if 'de-rate' in description.lower():
        category_flags['derate'] = True
    # check if 'upgrade' is present in the description
    if 'upgrade' in description.lower():
        category_flags['upgrade'] = True
    

    return category_flags

def extract_number_before_word(text, word):
    pattern = fr"(\d+(?:\.\d+)?)\s*{word}"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return float(match.group(1).strip().replace(',', '.'))
    return None
# OLD FUNCTIONS END HERE

# this is a simple function to mine for certain new upgrades. It can use any new set of upgrade-word pairs
# as defined in the 'new_product_categories' dictionary. Takes a single description, returns dictionary of upgrade flags.
def classify_product_new(description):
    if isinstance(description, float):
        description = str(description)
    # initialize a dictionary to store the boolean flags for each product category in the description
    category_flags = {category: 0 for category in new_product_categories}

    # check if any of the keywords for each product category is present in the description
    for category, keywords in new_product_categories.items():
        for keyword in keywords:
            if keyword.lower() in description.lower():
                category_flags[category] = 1

    return category_flags

First, permits are mined for specific upgrades with the code below

In [85]:
# apply the classify_product function to each description in the dataframe
df = pd.concat([df, pd.DataFrame(list(df['Description'].apply(classify_product)))], axis=1)
df.head()

,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,PropertyAddressAddressStreetSuffix,PropertyAddressAddressStreetPostDirection,...,Carbon Monoxide Detector,Ductwork,Gas Furnace,Dishwasher,Washing Machine,Dryer,Elevator,Skylight,upgrade,derate
0,22210890,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,False,False,False,False,NaN,NaN
1,492282549,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,False,False,False,False,NaN,NaN
2,457815823,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,False,False,False,False,True,NaN
3,461991399,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,False,False,False,False,NaN,NaN
4,476990183,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,False,False,False,False,NaN,NaN


In [86]:
# Extract numbers before 'kwh' and create a new column
df['kwH'] = df['Description'].apply(lambda x: extract_number_before_word(x, 'kwh'))

# Extract numbers before 'kw' and create a new column
df['kw'] = df['Description'].apply(lambda x: extract_number_before_word(x, 'kw'))

In [87]:
df['kwH_final'] = df.apply(lambda row: '' if row['Solar PV'] and row['kwH'] == row['kw'] else row['kwH'], axis=1)
df['kw_final'] = df.apply(lambda row: '' if row['Battery storage'] and row['kwH'] == row['kw'] else row['kw'], axis=1)
df.loc[df['Solar PV'] == False, 'kw'] = ''
df.loc[df['Battery storage'] == False, 'kwH'] = ''
df.head()

/var/folders/5h/45mckv054hbfvkst9pks_x800000gn/T/ipykernel_90805/4198677014.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Solar PV'] == False, 'kw'] = ''
/var/folders/5h/45mckv054hbfvkst9pks_x800000gn/T/ipykernel_90805/4198677014.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Battery storage'] == False, 'kwH'] = ''


,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,PropertyAddressAddressStreetSuffix,PropertyAddressAddressStreetPostDirection,...,Washing Machine,Dryer,Elevator,Skylight,upgrade,derate,kwH,kw,kwH_final,kw_final
0,22210890,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,NaN,NaN,,,NaN,NaN
1,492282549,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,NaN,NaN,,,NaN,NaN
2,457815823,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,True,NaN,,,NaN,NaN
3,461991399,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,NaN,NaN,,,NaN,NaN
4,476990183,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,False,False,NaN,NaN,,,NaN,NaN


In [88]:
df.to_csv('SDGEPermitDataRough.csv')

Code specifically for panel amp predictions starts here

In [107]:
Existing = ['from', 'existing']
Upgrade = ['to', 'new', 'upgrade', 'derate', 'downsized']

select = [Existing, Upgrade]
first = [[Existing[0]], [Upgrade[0:2]]]

def extract_numbers_AMP(text, word):
    pattern = fr"(\d+(?:\.\d+)?)\s*{word}"
    matches = re.findall(pattern, text, re.IGNORECASE)
    if matches:
        return [float(match.strip()) for match in matches]
    return None

def extract_all_numbers_before_A(text):
    pattern = r"(\d+(?:\.\d+)?)\s*A\b"
    matches = re.findall(pattern, text, re.IGNORECASE)
    if matches:
        return [float(match.strip()) for match in matches]
    return None

def get_number(value):
    for i in range(0, len(value)):
        if value[i] not in digits:
            return value[:i]
    return value

def amp_predictor(description, version):
    result = nan
    words = description.split()
    for i in range(0, len(words)):
        curr = words[i]
        if curr in select[version]:
            if curr in first[version] and i < len(words)-1:
                if words[i+1][0] in digits:
                    result = get_number(words[i+1])
            else:
                if i < len(words)-1:
                    if words[i+1][0] in digits:
                        result = get_number(words[i+1])
                elif i > 0:
                    if words[i-1][0] in digits:
                        result = get_number(words[i-1])
        
    return float(result)

def fill_predictions(row):
    options = row['combined_column']
    e = row['E']
    u = row['U']
    result = {'Existing Amp': e, 'Upgrade Amp': u}
    if options == '' or options == None:
        return result
    description = row['Description']
    if len(options) == 1:
        result['Upgrade Amp'] = options[0]
    if len(options) == 2 and 'upgrade' in description:
        if (options[0] == options[1]):
            result['Upgrade Amp'] = options[0]
        else:
            result['Upgrade Amp'] = max(options[0], options[1])
    return result

In [108]:
# Extract numbers before Amp, A, and create new columns
df['Amp_2'] = df['Description'].apply(lambda x: extract_numbers_AMP(x, 'Amp'))
df['A_2'] = df['Description'].apply(extract_all_numbers_before_A)
df['combined_column'] = df['Amp_2'].fillna(df['A_2'])
df.loc[df['Electrical Panel Upgrade'] == False, 'combined_column'] = ''
df.head()

,Unnamed: 0,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,PropertyAddressAddressStreetSuffix,...,kwH_final,kw_final,# Panel or Module,Use Code,Last Sale Date,Year Built,APN,Amp_2,A_2,combined_column
0,0,1027343064,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,AVE,...,NaN,NaN,NaN,1001.0,20220107.0,1951.0,4012029015,None,None,
1,1,1027343201,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,AVE,...,NaN,NaN,NaN,1001.0,20220107.0,1951.0,4012029015,None,None,
2,2,1030052530,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,AVE,...,NaN,NaN,NaN,1001.0,20220107.0,1951.0,4012029015,None,None,
3,3,457184115,16492599.0,Los Angeles,6037.0,28936 SAM PL,28936,NaN,SAM,PL,...,NaN,NaN,NaN,1004.0,20160920.0,1983.0,2837023037,None,None,
4,4,932568316,16492626.0,Los Angeles,6037.0,10409 WILSEY AVE,10409,NaN,WILSEY,AVE,...,NaN,NaN,NaN,1001.0,20181030.0,1924.0,2558010007,None,None,


In [109]:
df['E'] = df['Description'].apply(lambda x: amp_predictor(x, 0))
df['U'] = df['Description'].apply(lambda x: amp_predictor(x, 1))
df.loc[df['Electrical Panel Upgrade'] == False, 'E'] = nan
df.loc[df['Electrical Panel Upgrade'] == False, 'U'] = nan
df.head()

,Unnamed: 0,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,PropertyAddressAddressStreetSuffix,...,# Panel or Module,Use Code,Last Sale Date,Year Built,APN,Amp_2,A_2,combined_column,E,U
0,0,1027343064,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,AVE,...,NaN,1001.0,20220107.0,1951.0,4012029015,None,None,,NaN,NaN
1,1,1027343201,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,AVE,...,NaN,1001.0,20220107.0,1951.0,4012029015,None,None,,NaN,NaN
2,2,1030052530,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,AVE,...,NaN,1001.0,20220107.0,1951.0,4012029015,None,None,,NaN,NaN
3,3,457184115,16492599.0,Los Angeles,6037.0,28936 SAM PL,28936,NaN,SAM,PL,...,NaN,1004.0,20160920.0,1983.0,2837023037,None,None,,NaN,NaN
4,4,932568316,16492626.0,Los Angeles,6037.0,10409 WILSEY AVE,10409,NaN,WILSEY,AVE,...,NaN,1001.0,20181030.0,1924.0,2558010007,None,None,,NaN,NaN


In [110]:
amp_data = pd.DataFrame(list(df.apply(lambda x: fill_predictions(x), axis=1)))

df_result = pd.concat([df, amp_data], axis=1)
epu_result = df_result[df['Electrical Panel Upgrade'] == True]
epu_result.head()

,Unnamed: 0,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,PropertyAddressAddressStreetSuffix,...,Last Sale Date,Year Built,APN,Amp_2,A_2,combined_column,E,U,Existing Amp,Upgrade Amp
22,22,462680731,16492790.0,Los Angeles,6037.0,28013 DEEP CREEK DR,28013,NaN,DEEP CREEK,DR,...,20210208.0,1966.0,2844018012,[200.0],None,[200.0],NaN,200.0,NaN,200.0
29,29,1059125810,16492819.0,Los Angeles,6037.0,17138 SIMONDS ST,17138,NaN,SIMONDS,ST,...,20130702.0,1954.0,2697013014,None,[200.0],[200.0],NaN,NaN,NaN,200.0
35,35,1007406000,16492867.0,Los Angeles,6037.0,20920 COMMUNITY ST APT 14,20920,NaN,COMMUNITY,ST,...,20210928.0,1987.0,2779006111,None,[125.0],[125.0],NaN,NaN,NaN,125.0
36,36,942681329,16492874.0,Los Angeles,6037.0,27720 FOXLANE DR,27720,NaN,FOXLANE,DR,...,20200610.0,1962.0,2802014023,[200.0],None,[200.0],NaN,200.0,NaN,200.0
40,40,988379566,16492883.0,Los Angeles,6037.0,10850 GAYNOR AVE,10850,NaN,GAYNOR,AVE,...,20011016.0,1954.0,2667021017,None,"[200.0, 100.0]","[200.0, 100.0]",NaN,100.0,NaN,200.0


In [81]:
epu_result.to_csv('Panel upgrade data.csv')

In [111]:
clean_df = df_result.drop(columns=['Amp_2', 'A_2', 'combined_column', 'E', 'U'])
clean_df.head()

,Unnamed: 0,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,PropertyAddressAddressStreetSuffix,...,Skylight,kwH_final,kw_final,# Panel or Module,Use Code,Last Sale Date,Year Built,APN,Existing Amp,Upgrade Amp
0,0,1027343064,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,AVE,...,0,NaN,NaN,NaN,1001.0,20220107.0,1951.0,4012029015,NaN,NaN
1,1,1027343201,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,AVE,...,0,NaN,NaN,NaN,1001.0,20220107.0,1951.0,4012029015,NaN,NaN
2,2,1030052530,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,AVE,...,0,NaN,NaN,NaN,1001.0,20220107.0,1951.0,4012029015,NaN,NaN
3,3,457184115,16492599.0,Los Angeles,6037.0,28936 SAM PL,28936,NaN,SAM,PL,...,0,NaN,NaN,NaN,1004.0,20160920.0,1983.0,2837023037,NaN,NaN
4,4,932568316,16492626.0,Los Angeles,6037.0,10409 WILSEY AVE,10409,NaN,WILSEY,AVE,...,0,NaN,NaN,NaN,1001.0,20181030.0,1924.0,2558010007,NaN,NaN


In [112]:
clean_df.to_csv('Permit Mining Results Sept2024.csv')

Simple version if you only want to add new upgrades or upgrades with expanded word lists:

In [6]:
df['Description'] = df['Description'].astype(str)

# splitting dataframe so the data we're adding goes in the right spot
front = df[columns[:62]]
back = df[columns[62:]]

# goes through every description, mines upgrades, then takes all results and makes new DF
insert = pd.DataFrame(list(df['Description'].apply(classify_product_new)))

# putting everything together for new permit data result
new_df = pd.concat([front, insert, back], axis=1)
new_df.head()

,Unnamed: 0.1,Unnamed: 0,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,...,Washing Machine,Dryer,Elevator,Skylight,combined_column,kwH_final,kw_final,Existing Amp,Upgrade Amp,# Panel or Module
0,0,0,1027343064,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1027343201,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,1030052530,16492574.0,Los Angeles,6037.0,7230 BRYNHURST AVE,7230,NaN,BRYNHURST,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,457184115,16492599.0,Los Angeles,6037.0,28936 SAM PL,28936,NaN,SAM,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,932568316,16492626.0,Los Angeles,6037.0,10409 WILSEY AVE,10409,NaN,WILSEY,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
new_df.to_csv('filename.csv')

The next section is to add property data + use codes which makes the process of splitting the data by property and then by use codes significantly easier. Also makes calculating age, years since sale, etc. all easier.

Functions

In [96]:
# Function to connect all address parts into a single string for comparison since
# permit data contains full addresses but use code data does not. Takes row (property) as input,
# returns address (in lowercase) as a string.
def addr_full(row):
    address = ''
    number = str(row['SITE_HOUSE_NUMBER'])
    direction = str(row['SITE_DIRECTION'])
    street = str(row['SITE_STREET_NAME'])
    mode = str(row['SITE_MODE'])
    if number != 'nan':
        index = number.find('.')
        if index != -1:
            number = number[:index]
        address += number
    if direction != 'nan':
        address = address + ' ' + direction
    if street != 'nan':
        address = address + ' ' + street
    if mode != 'nan':
        address = address + ' ' + mode
    return address.lower()

# Function to handle addresses containing unit numbers, which aren't in the use code data (all units kept together)
# Takes permit and constructs partial address, returning it as a string
def addr_partial(row):
    address = ''
    number = str(row['PropertyAddressAddressHouseNumber'])
    direction = str(row['PropertyAddressAddressStreetDirection'])
    street = str(row['PropertyAddressAddressStreetName'])
    s_suffix = str(row['PropertyAddressAddressStreetSuffix'])
    spd = str(row['PropertyAddressAddressStreetPostDirection'])
    if number != 'nan':
        index = number.find('.')
        if index != -1:
            number = number[:index]
        address += number
    if direction != 'nan':
        address = address + ' ' + direction
    if street != 'nan':
        address = address + ' ' + street
    if s_suffix != 'nan':
        address = address + ' ' + s_suffix
    if spd != 'nan':
        address = address + ' ' + spd
    return address.lower()

# Builds dictionary of property -> property data pairs
def build_dict(pathname, prop_dict, version):
    county_df = pd.read_csv(pathname)
    for index, row in county_df.iterrows():
        if version == 0:
            address = addr_full(row)
            prop_dict[address] = row
        elif version == 1:
            prop_dict[row['MASTER_PARCEL_APN']] = row

# gets property data from dictionary, with an addtional check to get rid of unit/apartment numbers. 
# uncomment use code lines & add back to 'cat' list, commented  out
# because they were already added using an earlier version of this function.
def get_prop_data(row, propdict):
    cat = ['Last Sale Date', 'Year Built', 'APN']
    prop_data = {i: '' for i in cat}
    key = row['PropertyAddressFull'].lower()
    if key in propdict.keys():
        prop_data['Use Code'] = str(propdict[key]['USE_CODE_STD_LPS'])
        prop_data['Last Sale Date'] = propdict[key]['LAST_SALE_DATE_TRANSFER']
        prop_data['Year Built'] = propdict[key]['YR_BLT']
        prop_data['APN'] = str(propdict[key]['MASTER_PARCEL_APN'])
    else:
        key = addr_partial(row)
        if key in propdict.keys():
            prop_data['Use Code'] = str(propdict[key]['USE_CODE_STD_LPS'])
            prop_data['Last Sale Date'] = propdict[key]['LAST_SALE_DATE_TRANSFER']
            prop_data['Year Built'] = propdict[key]['YR_BLT']
            prop_data['APN'] = str(propdict[key]['MASTER_PARCEL_APN'])
    return prop_data

Building dictionary of property -> property data (required since the property data files are all for individual counties)

In [5]:
# this is for the case with the old ATTOM data, multiple counties

path = 'use_code_data/*.csv' # replace with own path, only works if all use code data is in one folder with no other .csv files

prop_dict = {}

key_type = 0
for pathname in glob.glob(path):
    build_dict(pathname, prop_dict, key_type)

/var/folders/5h/45mckv054hbfvkst9pks_x800000gn/T/ipykernel_72435/3538860228.py:51: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  county_df = pd.read_csv(pathname)
/var/folders/5h/45mckv054hbfvkst9pks_x800000gn/T/ipykernel_72435/3538860228.py:51: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  county_df = pd.read_csv(pathname)
/var/folders/5h/45mckv054hbfvkst9pks_x800000gn/T/ipykernel_72435/3538860228.py:51: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  county_df = pd.read_csv(pathname)
/var/folders/5h/45mckv054hbfvkst9pks_x800000gn/T/ipykernel_72435/3538860228.py:51: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  county_df = pd.read_csv(pathname)
/var/folders/5h/45mckv054hbfvkst9pks_x800000gn/T/ipykernel_72435/3538860228.py:51: DtypeWarning: Columns (3,4) have mixed ty

In [97]:
# for SDGE data, all San Diego county

prop_dict = {}

build_dict('sandiego.csv', prop_dict, 0)

/var/folders/5h/45mckv054hbfvkst9pks_x800000gn/T/ipykernel_90805/646359303.py:49: DtypeWarning: Columns (4,27) have mixed types. Specify dtype option on import or set low_memory=False.
  county_df = pd.read_csv(pathname)


Getting property data

In [98]:
df = clean_df # or load from rough.csv
df.head()

,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,PropertyAddressAddressStreetSuffix,PropertyAddressAddressStreetPostDirection,...,Elevator,Skylight,upgrade,derate,kwH,kw,kwH_final,kw_final,Existing Amp,Upgrade Amp
0,22210890,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,NaN,NaN,,,NaN,NaN,NaN,NaN
1,492282549,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,NaN,NaN,,,NaN,NaN,NaN,NaN
2,457815823,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,True,NaN,,,NaN,NaN,NaN,NaN
3,461991399,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,NaN,NaN,,,NaN,NaN,NaN,NaN
4,476990183,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,False,False,NaN,NaN,,,NaN,NaN,NaN,NaN


In [99]:
df['PropertyAddressFull'] = df['PropertyAddressFull'].astype(str)

# goes through each permit and pulls the property data from the dictionary
df_prop_data = pd.DataFrame(list(df.apply(lambda x: get_prop_data(x, prop_dict), axis=1)))

# result
new_df = pd.concat([df, df_prop_data], axis=1)
new_df.head()

,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,PropertyAddressAddressStreetSuffix,PropertyAddressAddressStreetPostDirection,...,kwH,kw,kwH_final,kw_final,Existing Amp,Upgrade Amp,Last Sale Date,Year Built,APN,Use Code
0,22210890,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,,,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001
1,492282549,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,,,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001
2,457815823,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,,,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001
3,461991399,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,,,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001
4,476990183,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,,,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001


In [27]:
clean_df = new_df.drop(columns=list(new_df)[0:1])

clean_df.head()

,BuildingPermitID,[ATTOM ID],CountyName,StateCountyFIPS,PropertyAddressFull,PropertyAddressAddressHouseNumber,PropertyAddressAddressStreetDirection,PropertyAddressAddressStreetName,PropertyAddressAddressStreetSuffix,PropertyAddressAddressStreetPostDirection,...,A_2,kwH,kw,combined_column,kwH_final,kw_final,Last Sale Date,Year Built,APN,Use Code
0,22210890,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001
1,492282549,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001
2,457815823,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001
3,461991399,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001
4,476990183,204042384.0,San Diego,6073.0,31510 AQUADUCT RD,31510,NaN,AQUADUCT,RD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20210908.0,1945.0,1271107900,1001


In [103]:
new_df.to_csv('Permit Mining Results SDGE.csv')

In [102]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016880 entries, 0 to 2016879
Data columns (total 87 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   BuildingPermitID                            int64  
 1   [ATTOM ID]                                  float64
 2   CountyName                                  object 
 3   StateCountyFIPS                             float64
 4   PropertyAddressFull                         object 
 5   PropertyAddressAddressHouseNumber           object 
 6   PropertyAddressAddressStreetDirection       object 
 7   PropertyAddressAddressStreetName            object 
 8   PropertyAddressAddressStreetSuffix          object 
 9   PropertyAddressAddressStreetPostDirection   object 
 10  PropertyAddressAddressUnitPrefix            object 
 11  PropertyAddressAddressUnit                  object 
 12  PropertyAddressCity                         object 
 13  PropertyAddressState       